In [1]:
from ib_insync import *  #use ibkr api to trade
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display, clear_output
util.startLoop() 


In [2]:
import predict_data as mod

[*********************100%%**********************]  1 of 1 completed


1/1 [==============================] - 0s 266ms/step
[[179.07199]
 [179.03308]
 [177.29698]
 [175.34393]
 [175.32478]
 [174.93161]
 [176.44063]
 [177.79567]]


In [3]:
pred_price = mod.predict_data("DHI", 40, 8) #get today close price
tdy_close_price = pred_price.tolist()[0][0]
tdy_close_price

[*********************100%%**********************]  1 of 1 completed


1/1 [==============================] - 0s 192ms/step


179.07199096679688

In [4]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [5]:
contract = Stock(symbol='DHI',exchange = "SMART")
ib.qualifyContracts(contract)
conID = contract.conId

In [6]:
trading_units = 1000
end_time = dt.time(23, 59, 0)

In [7]:
#ema_s = 5
#ema_l = 8

In [8]:
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    if bars[-1].date > last_bar: # if bar completed / new bar
        last_bar = bars[-1].date
    
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
                
        df['position'] = np.where(df.close < tdy_close_price, 1, -1)
        target = df['position'][-1] * trading_units

        
        clear_output(wait=True)
        display(df)
     
    else:
        clear_output(wait=True)
        trade_reporting()

In [9]:
def trade_execute(target): 
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conId][0]
    except:
        current_pos = 0
        
    trades = target - current_pos
    print(trades,target,current_pos)
    if trades < 0:
        side = 'SELL'
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(contract, order)
        
        
    elif trades > 0:
        side = 'BUY'
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(contract, order)
    else: 
        pass

In [10]:
def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
    
    clear_output(wait=True)
    display(report)
    
    

In [ ]:
session_start = pd.to_datetime(dt.datetime.utcnow()).tz_localize("utc")
bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting='10 mins',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)
ib.sleep(10)
if bars:
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate
else:
    print("No historical data available.")
bars.updateEvent += onBarUpdate

# stop trading session
while True:
    ib.sleep(5) # check every 5 seconds
    if dt.datetime.utcnow().time() >= end_time: # if stop conditions has been met
        trade_execute(target = 0) # close open position 
        ib.cancelHistoricalData(bars) # stop stream
        ib.sleep(10)
        try:
            trade_reporting() # final reporting
        except:
            pass
        print("Session Stopped.")
        ib.disconnect()
        break
    else:
        pass

In [ ]:
ib.disconnect()


In [ ]:
ib.isConnected